# Extracción de datos de la base de datos de PTB Diagnostic ECG Database

In [1]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks

/home/gino/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
folder_name = './data/autonomic-aging-a-dataset'
files = []
diagnosis_per_patient = {}
kinds_of_diagnosis = set()
patients_per_diagnosis = {}

In [3]:
def bandpass_filter(signal, lowcut, highcut, fs, order=1):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, signal)

In [4]:
def calculate_hr(p_record_name):
    try:
        record = wfdb.rdrecord(p_record_name)
        ecg_data = record.p_signal  # Load all 12-lead ECG signals
    
        # Step 2: Select Lead II for Heart Rate Calculation
        lead_II = ecg_data[:, 1]  # Assuming lead II is the second column
        
        fs = 1000  # Sampling frequency is 1000 Hz
        filtered_lead_II = bandpass_filter(lead_II, 0.5, 50, fs)
        
        # Step 4: R-Peak Detection
        # Use the find_peaks function to detect R-peaks
        peaks, _ = find_peaks(filtered_lead_II, distance=fs*0.6)  # Assuming a minimum distance of 600ms between peaks
        
        # Step 5: Calculate RR Intervals and Heart Rate
        rr_intervals = np.diff(peaks) / fs  # RR intervals in seconds
        hr_values = 60 / rr_intervals  # Heart rate in beats per minute (bpm)
        
        # Step 6: Generate Time Axis for Heart Rate Plot
        time_peaks = peaks / fs  # Time of R-peaks in seconds
        time_hr = (time_peaks[:-1] + time_peaks[1:]) / 2  # Midpoint between successive peaks
        return time_hr, hr_values
    except Exception as e:
        print(f"Error processing record {p_record_name}: {e}")
        return [], []

In [5]:
def plot_hr_signal(time_hr, hr_values):
    plt.figure(figsize=(10, 6))
    plt.plot(time_hr, hr_values, label='Heart Rate (bpm)', color='b', marker='o', linestyle='-')
    plt.title('Heart Rate over Time')
    plt.xlabel('Time (s)')
    plt.ylabel('Heart Rate (bpm)')
    plt.grid(True)
    plt.legend()
    plt.show()

In [6]:
# Descomentar para visualizar métricas de diagnóstico
# with open(folder_name + '/RECORDS') as f:
#     for line in f:
#         record_name = line.strip()
#         file_name = f'{folder_name}/{record_name}'
#         files.append(file_name)
#         header = wfdb.rdheader(file_name)
#         diagnosis = header.comments[4].split(': ')[1]
#         diagnosis_per_patient[f'{record_name}'] = diagnosis
#         kinds_of_diagnosis.add(diagnosis)
#         if diagnosis in patients_per_diagnosis:
#             patients_per_diagnosis[diagnosis].append(record_name)
#         else:
#             patients_per_diagnosis[diagnosis] = [record_name]
#         time_hr, hr_values = calculate_hr(file_name)
#         plot_hr_signal(time_hr, hr_values)



In [7]:
import pandas as pd
from scipy.signal import find_peaks
from hr_engine import get_features


times = pd.DataFrame()
hear_rates = pd.DataFrame()
process_only_one = False
results = []
with open(folder_name + '/RECORDS') as f:
    for line in f:
        
        record_name = line.strip()
        file_name = f'{folder_name}/{record_name}'
        files.append(file_name)
        header = wfdb.rdheader(file_name)
        # diagnosis = header.comments[4].split(': ')[1]
        # if diagnosis == 'n/a':
        #     continue
        # diagnosis_per_patient[f'{record_name}'] = diagnosis
        # kinds_of_diagnosis.add(diagnosis)
        # if diagnosis in patients_per_diagnosis:
        #     patients_per_diagnosis[diagnosis].append(record_name)
        # else:
        #     patients_per_diagnosis[diagnosis] = [record_name]
        time_hr, hr_values = calculate_hr(file_name)
        
        # remove first 5 and last 5 values of time_hr and hr_values
        time_hr = time_hr[5:-5]
        hr_values = hr_values[5:-5]

        # if diagnosis == 'n/a':
        #     continue
        
        # diagnosis = 0 if diagnosis == "Healthy control" else 1
        # print(time_hr)
        if(len(hr_values) == 0 or len(time_hr) == 0):
            continue
        results.append(get_features(hr_values=hr_values, time_hr=time_hr, record_name=record_name, diagnosis=0))

        if process_only_one:
            break
# print(kinds_of_diagnosis)
df = pd.DataFrame(results)
df.head()

# print(kinds_of_diagnosis)
        

/home/gino/Documents/Repos/pia-02/hr_engine.py:101: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hr_smoothed = pd.Series(hr_values).rolling(window=window_size, center=True).mean().fillna(method='bfill').fillna(
/home/gino/Documents/Repos/pia-02/hr_engine.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.replace([np.inf, -np.inf], np.nan, inplace=True)
/home/gino/Documents/Repos/pia-02/hr_engine.py:101: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hr_smoothed = pd.Series(hr_values).rolling(window=window_size, center=True).mean().fillna(method='bfill').fillna(
/home/gino/Documents/Repos/pia-02/hr_engine.py:158: Set

Error processing record ./data/autonomic-aging-a-dataset/0400: [Errno 2] No such file or directory: '/home/gino/Documents/Repos/pia-02/data/autonomic-aging-a-dataset/0400.dat'


/home/gino/Documents/Repos/pia-02/hr_engine.py:101: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hr_smoothed = pd.Series(hr_values).rolling(window=window_size, center=True).mean().fillna(method='bfill').fillna(
/home/gino/Documents/Repos/pia-02/hr_engine.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.replace([np.inf, -np.inf], np.nan, inplace=True)
/home/gino/Documents/Repos/pia-02/hr_engine.py:101: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hr_smoothed = pd.Series(hr_values).rolling(window=window_size, center=True).mean().fillna(method='bfill').fillna(
/home/gino/Documents/Repos/pia-02/hr_engine.py:158: Set

,minimum_hrv,maximum_hrv,standard_deviation_hrv,mean_hr_slope,tendency_slope,lowest_heart_rate,vlf_power,lf_power,hf_power,approximation_entropy,diagnosis
0,0.074156,23.767573,3.473864,-0.010159,0.001633,0.550964,9.806508,106.363675,345.201028,1.712355,0.0
1,0.055527,42.127947,3.449033,0.005524,0.000977,0.500417,34.234365,295.876823,1138.215221,1.813090,0.0
2,0.135882,2.286585,0.478068,0.000882,-0.000044,0.891530,2.321808,26.653556,224.175681,2.161885,0.0
3,0.051014,45.206571,3.176905,-0.011961,-0.001033,0.511945,8.712358,74.331849,300.399533,1.644848,0.0
4,0.057165,40.957202,7.312454,0.017386,0.004113,0.494641,11.539303,100.524827,579.162122,1.437170,0.0


In [8]:
df.to_csv('autonomic-aging-a-dataset-clear-data.csv', index=False)